# **Practice 7**

1. In this practice we are going to program and use an explicit and an implicit method to solve an ODE.

+ Based on explicit Euler's method provided in the class' notebook program Heun's method.

+ Based on that same code and the Newton's method that you coded in practice 3, program the implicit Euler's method.

+ Solve the initial value 
$$y'=-\frac{3t^2y+y^2}{2t^3+3ty}$$
$$ y(1)=-2$$
on the interval $[1,2]$. Use step sizes $h = 0.0001, 0.001, 0.01, 0.1$. 

+ Use the implicit solution $t^3y^2+ty^3+4=0$ to check your results.

**Note:** Try to write functions that you could use for other problems with small changes, if any.

# Solution

### Imports

In [21]:
import inspect
import sys
from typing import Callable

import numpy as np
import scipy.special as sci

### Newton's method

In [22]:
def forward(order: int, f: 'Callable', x: float, h: float, exact: bool = False) -> float:
    """Use forward finite difference formula of order `order` to compute the derivative of `f` at `x`.

    Args:
        order (int): Order of the derivate. (first, second ...).
        f (Callable): Function for which the derivative will be computed.
            Only one argument will be passed to it, `x`.
        x (float): Point at which the derivative will be computed.
        h (float): Error of the approximation # TODO: Is it the error?
        exact (bool, optional): Set the precision of the method.
            If False, floating comma numbers are used. Set to True to use long numbers. Defaults to False.

    Note:
        See more of `exact` parameter in https://docs.scipy.org/doc/scipy/reference/generated/scipy.special.comb.html

    Returns:
        float: Derivative of order `order` of `f` evaluated at `x`.
    """
    return sum([(-1)**(order-k) * sci.comb(order, k, exact) * f(x + k*h) for k in range(order+1)]) / h**order


def newton(err: float = 1e-4, f: 'Callable[float]' = None, f_dev: 'Callable[float]' = None, 
           differentiator: 'Callable[int, Callable, float, float, bool]' = None, *,
           x0: float = 0, h_err: float = 1e-1) -> float:
    r"""Newton's method to find roots of a function.

    Args:
        err (float): Desired error of the method.
        f (Callable[float], optional): Analytical function to find its roots. Its input is the point to be evaluated in. Defaults to None.
        f_dev (Callable[float], optional): Analytical derivative of the function. Its input is the point to be evaluated in. Defaults to None.
        x0 (float, optional): Initial guess of the root.
            Note that an inadequate first guess could lead to undesired outputs such as no roots or undesired roots.
            Defaults to 0.
        
    Returns:
        float|None: Root of the function or None if the algorithm reaches its recursion limit.
    """
    iter, iter_dict = 0, {0: x0} # Not necessary to store all the values, but anyway
    limit = sys.getrecursionlimit()

    while True:
        if iter + 10 >= limit:
            print(f'Iteration limit ({limit}) reached without finding any root. Try with other initial guess or changing the recursion limit. \
                    Maybe there are no roots.')
            return

        print(differentiator(1, f, iter_dict[iter], h_err, True), '\t', iter_dict[iter])
        iter_dict[iter+1] = iter_dict[iter] - f(iter_dict[iter]) / differentiator(1, f, iter_dict[iter], h_err, True)

        if abs(iter_dict[iter+1] - iter_dict[iter]) < err:
            return iter_dict[iter+1]

        iter += 1

## Exercise 1

### Implementation of methods

In [23]:
def euler_explicit(f: 'Callable[float, float]', y0: float, t0: float, t: float, h: float) -> np.ndarray:
    r"""Computes the explicit (forward) Euler method to solve ODEs.

    Args:
        f (Callable[float, float]): Function depending on y and t in that order.
            Equivalent to f(y,t).
        y0 (float): Initial value of the answer.
            Equivalent to y(t0).
        t0 (float): Initial time.
        t (float): Final time.
        h (float): Separation between the points of the interval.

    Returns:
        np.ndarray: Numerical solution of the ODE in the interval [t0, t0+h, ..., t-h, t].
    """
    t_ = np.arange(t0, t+h, h)
    N = len(t_)

    u = np.zeros_like(t_)
    u[0] = y0

    for i in range(N-1):
        u[i+1] = u[i] + h * f(u[i], t_[i])

    return u


def heun(f: 'Callable[float, float]', y0: float, t0: float, t: float, h: float) -> np.ndarray:
    """Computes Heun's method to solve ODEs.

    Args:
        f (Callable[float, float]): Function depending on y and t in that order.
            Equivalent to f(y,t).
        y0 (float): Initial value of the answer.
            Equivalent to y(t0).
        t0 (float): Initial time.
        t (float): Final time.
        h (float): Separation between the points of the interval.

    Returns:
        np.ndarray: Numerical solution of the ODE in the interval [t0, t0+h, t-h, t].
    """
    t_ = np.arange(t0, t+h, h)
    N = len(t_)

    u = np.zeros_like(t_)
    u[0] = y0

    for i in range(N-1):
        u[i+1] = u[i] + h/2 * (f(u[i]+h*f(u[i], t_[i]), t_[i+1]) + f(u[i], t_[i]))

    return u


def euler_implicit(f: 'Callable[float, float]', y0: float, t0: float, t: float, h: float, *args, **kwargs) -> np.ndarray:
    """Computes the implicit (backward) Euler method to solve ODEs.

    If `f` argument has an explicit dependence on y, Newton\' method is used to compute the next iteration the algorithm.
    Then, `err` must be passed as extra argument, and it is recommended to pass the analytical derivative of `f` as f_dev.
    In case this last `f_dev` is not passed, Newton\' method will use finite differences to numerically obtain it.

    See more about Newton\' method in module roots.

    Args:
        f (Callable[float, float]): Function depending on y and t in that order.
            Equivalent to f(y,t).
        y0 (float): Initial value of the answer.
            Equivalent to y(t0).
        t0 (float): Initial time.
        t (float): Final time.
        h (float): Separation between the points of the interval.

    Returns:
        np.ndarray: Numerical solution of the ODE in the interval [t0, t0+h, ..., t-h, t].
    """
    t_ = np.arange(t0, t+h, h)
    N = len(t_)

    u = np.zeros_like(t_)
    u[0] = y0

    for i in range(N-1):
        def g(y): return u[i] - u[i+1] + h*f(y, t_[i+1])
        u[i+1] = newton(f=g, differentiator=forward, x0=u[i], *args, **kwargs)

    return u

### Solving the IVP

For it to work, write the analytical derivate. Im too lazy.

In [24]:
def f_t_y(y, t):
    return - (3* t**2 * y + y**2) / (2* t**3 + 3* t*y)

h_vec = [0.0001, 0.001, 0.01, 0.1]

for hi in h_vec:

    print(f"Euler explicit wit h={hi} yields {euler_explicit(f_t_y, -2, 1, 2, hi)}")
    print(f"Euler implicit wit h={hi} yields {euler_implicit(f_t_y, -2, 1, 2, hi)}")        
    

-6.487755756179325e-05 	 -2.0
-3.333000039074108e-05 	 -30830.072220463655
-3.3330000352993494e-05 	 -60008.333800214445
-3.332665073685348e-05 	 -60008.33382595631
-3.33265052177012e-05 	 -1800610999.7833
-3.332672349642962e-05 	 -1800610067.111356
-3.332665073685348e-05 	 -1800610067.1159112
-3.337860107421875e-05 	 -1800610067.1159115
-3.5762786865234375e-05 	 -53945045888007.88
-3.5762786865234375e-05 	 -54028405150041.125
-3.5762786865234375e-05 	 -54034091250426.625
-3.0994415283203125e-05 	 -54034479110597.234
-3.0994415283203125e-05 	 -54034509637570.92
-3.5762786865234375e-05 	 -54034507343773.445
-3.5762786865234375e-05 	 -54034507493148.625
-3.0994415283203125e-05 	 -54034507503337.805
-3.5762786865234375e-05 	 -54034507504139.766
-3.5762786865234375e-05 	 -54034507504087.53
-3.5762786865234375e-05 	 -54034507504083.97
-3.337860107421875e-05 	 -54034507504083.73
-3.337860107421875e-05 	 -54034507504083.72
-3.5762786865234375e-05 	 -54034507504083.71
0.0 	 -54034507504083.71


/tmp/ipykernel_15439/1927144689.py:48: RuntimeWarning: divide by zero encountered in double_scalars
  iter_dict[iter+1] = iter_dict[iter] - f(iter_dict[iter]) / differentiator(1, f, iter_dict[iter], h_err, True)
/tmp/ipykernel_15439/2172801511.py:2: RuntimeWarning: invalid value encountered in double_scalars
  return - (3* t**2 * y + y**2) / (2* t**3 + 3* t*y)


nan 	 nan
nan 	 nan
nan 	 nan
nan 	 nan
nan 	 nan
nan 	 nan
nan 	 nan
nan 	 nan
nan 	 nan
nan 	 nan
nan 	 nan
nan 	 nan
nan 	 nan
nan 	 nan
nan 	 nan
nan 	 nan
nan 	 nan
nan 	 nan
nan 	 nan
nan 	 nan
nan 	 nan
nan 	 nan
nan 	 nan
nan 	 nan
nan 	 nan
nan 	 nan
nan 	 nan
nan 	 nan
nan 	 nan
nan 	 nan
nan 	 nan
nan 	 nan
nan 	 nan
nan 	 nan
nan 	 nan
nan 	 nan
nan 	 nan
nan 	 nan
nan 	 nan
nan 	 nan
nan 	 nan
nan 	 nan
nan 	 nan
nan 	 nan
nan 	 nan
nan 	 nan
nan 	 nan
nan 	 nan
nan 	 nan
nan 	 nan
nan 	 nan
nan 	 nan
nan 	 nan
nan 	 nan
nan 	 nan
nan 	 nan
nan 	 nan
nan 	 nan
nan 	 nan
nan 	 nan
nan 	 nan
nan 	 nan
nan 	 nan
nan 	 nan
nan 	 nan
nan 	 nan
nan 	 nan
nan 	 nan
nan 	 nan
nan 	 nan
nan 	 nan
nan 	 nan
nan 	 nan
nan 	 nan
nan 	 nan
nan 	 nan
nan 	 nan
nan 	 nan
nan 	 nan
nan 	 nan
nan 	 nan
nan 	 nan
nan 	 nan
nan 	 nan
nan 	 nan
nan 	 nan
nan 	 nan
nan 	 nan
nan 	 nan
nan 	 nan
nan 	 nan
nan 	 nan
nan 	 nan
nan 	 nan
nan 	 nan
nan 	 nan
nan 	 nan
nan 	 nan
nan 	 nan
nan 	 nan


KeyboardInterrupt: 